# Muon
## Checking events of some backgrounds after all cuts after the annealing method of BKG reduction
## Checking the events after a rotation
## the rotation is defined by the signal and is calculated based on the covariance matrix and the calculation of its eigenvalues.

In [1]:
import glob

In [2]:
import ROOT

Welcome to JupyROOT 6.24/06


In [3]:
samples_files = 'b'

BKG samples to be checked

In [4]:
mmg = glob.glob(f"../samples_400{samples_files}/*_mumu*_*.root")
ss = glob.glob(f"../samples_400{samples_files}/*_ssbar*_*.root")
dd = glob.glob(f"../samples_400{samples_files}/*_ddbar*_*.root")
uu = glob.glob(f"../samples_400{samples_files}/*_uubar*_*.root")
cc = glob.glob(f"../samples_400{samples_files}/*_ccbar*_*.root")

In [5]:
cortes_MC='visibleEnergyOfEventCMS < 10'
#and other cuts that were here

In [6]:
samples_name=[mmg, ss, dd, uu, cc]
nombre=['mmg', 'ss', 'dd', 'uu', 'cc']

Tree name, we will use RDataFrame

In [7]:
treename = "treevpho"

In [8]:
ruidos = {}

In [9]:
for i,n in enumerate(nombre):
    try:
        ruidos[n]={}
        ruidos[n]['df'] = ROOT.RDataFrame(treename,samples_name[i]) 
        ruidos[n]['pesos'] = 1
        print("cargado",n)
        #print(n, "ruido cargado",globals()[n])
    except Exception as e:
        print('---->',n)
        print(e,'\n\n')
        del ruidos[n]

cargado mmg
cargado ss
cargado dd
cargado uu
cargado cc


In [10]:
new_ruidos={}

Let's Filter or data frames for the given cuts and save the final events in a new dictionary new_ruidos as Numpy arrays

In [11]:
for ruido in ruidos:
    print(ruido,"\t --->")
    df_filter=ruidos[ruido]["df"].Filter(cortes_MC)
    
    count=df_filter.Count()
    val_events = count.GetValue()
    print(val_events)
    if val_events == 0:
        continue
    new_ruidos[ruido]=df_filter.AsNumpy()

mmg 	 --->
9
ss 	 --->
1
dd 	 --->
0
uu 	 --->
9
cc 	 --->
0


input_line_99:10:7: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
      ((const vector<double>*)obj)->empty();
      ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
input_line_114:10:7: warning: ignoring return value of function declared with 'nodiscard' attribute [-Wunused-result]
      ((const vector<int>*)obj)->empty();
      ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


Open the signal sample

In [12]:
cut_signalid='tau_sig_isSignal == 1'
signal = glob.glob(f"../samples_400c/tb_signal_muon.root")

In [13]:
ruidos['signal'] = { 
    'pesos': 1.0,
    'color': 'black',
    'df': ROOT.RDataFrame(treename, signal),
    #'style':1
}

Let's apply the same cuts for the signal

In [14]:
df_signal_base=ruidos['signal']['df'].Filter(cortes_MC)
df_signal = df_signal_base.Filter(cut_signalid)

Let's check the number of final events and efficiency

In [15]:
df_signal.Count().GetValue(),df_signal.Count().GetValue()*100/2000000

(170441, 8.52205)

In [16]:
#Let's save the final events in as Numpy arrays
numpy_df_signal=df_signal.AsNumpy()

In [17]:
import numpy as np

## we will start the calculation of the rotation of the events on the variables X and Y

In [18]:
xvar=numpy_df_signal['X']
yvar=numpy_df_signal['Y']
xvar_m=xvar-np.mean(xvar)
yvar_m=yvar-np.mean(yvar)

In [19]:
mcov=np.cov(xvar_m,yvar_m)
print(mcov)
mcor=np.corrcoef(xvar_m,yvar_m)
print(mcor)
e, v = np.linalg.eigh(mcov)
print(e,v)
order = e.argsort()[::-1]
print(order)
print(e[order])
v=v[:,order]
print(*v[:,0][::-1])
angle = np.degrees(np.arctan2(*v[:,0][::-1]))
print(angle)
yvar_m=np.array(yvar_m)
xvar_m=np.array(xvar_m)
#let's do the dot product
xs = np.dot(v.T, [xvar_m,yvar_m])
D=np.diag(1/(np.sqrt(e[order])))
#again for normalization
prod=np.dot(D,v.T)
xs_with=np.dot(prod, [xvar_m,yvar_m])

[[0.00072816 0.00205702]
 [0.00205702 0.01023342]]
[[1.        0.7535541]
 [0.7535541 1.       ]]
[0.0003021  0.01065947] [[-0.97921617  0.20281937]
 [ 0.20281937  0.97921617]]
[1 0]
[0.01065947 0.0003021 ]
0.979216167492851 0.20281937116708784
78.29812319036661


Let's make the products for the other samples

In [20]:
for element in new_ruidos:
    print(element)
    xvar_e=new_ruidos[element]['X']
    yvar_e=new_ruidos[element]['Y']
    xvar_e_m=xvar_e-np.mean(xvar)
    yvar_e_m=yvar_e-np.mean(yvar)
    yvar_e_m=np.array(yvar_e_m)
    xvar_e_m=np.array(xvar_e_m)
    xs_with_e=np.dot(prod, [xvar_e_m,yvar_e_m])
    new_ruidos[element]['rotvar_x']=xs_with_e[0,:]
    new_ruidos[element]['rotvar_y']=xs_with_e[1,:]

mmg
ss
uu


Final check of the invents after a salection in diferents regions, defined after the rotation

In [21]:
for sigma in [1,1.5,2,2.5,3]:
    print('---', sigma)
    for el in new_ruidos:
        #if el=="signal":
        #    continue
        #print(el,plot2d[el]['rotvar_x'])
        #print(el,plot2d[el]['rotvar_y'])
        a2=[k*k for k in new_ruidos[el]['rotvar_x']]
        b2=[k*k for k in new_ruidos[el]['rotvar_y']]
        #print('x',a2)
        #print('y',b2)
        suma=[sum(x) for x in zip(a2, b2)]
        #print('----')
        #print(suma)

        r2=sigma**2
        #print('---', r2)
        final_x=[]
        final_y=[]
        for j,ele in enumerate(suma):
            if ele < r2:
                #print('final',a2[j], b2[j])
                final_x.append(new_ruidos[el]['rotvar_x'][j])
                final_y.append(new_ruidos[el]['rotvar_y'][j])
        print('sample', el, ',', len(a2), f', {sigma} sigmas:', len(final_x) )
        #print(len(final_x)*100/2000000 )
        #if final_x:
        new_ruidos[el]['rotvar_x_final']=final_x
        new_ruidos[el]['rotvar_y_final']=final_y

    #break

--- 1
sample mmg , 9 , 1 sigmas: 0
sample ss , 1 , 1 sigmas: 0
sample uu , 9 , 1 sigmas: 3
--- 1.5
sample mmg , 9 , 1.5 sigmas: 0
sample ss , 1 , 1.5 sigmas: 0
sample uu , 9 , 1.5 sigmas: 3
--- 2
sample mmg , 9 , 2 sigmas: 1
sample ss , 1 , 2 sigmas: 0
sample uu , 9 , 2 sigmas: 3
--- 2.5
sample mmg , 9 , 2.5 sigmas: 2
sample ss , 1 , 2.5 sigmas: 0
sample uu , 9 , 2.5 sigmas: 4
--- 3
sample mmg , 9 , 3 sigmas: 3
sample ss , 1 , 3 sigmas: 0
sample uu , 9 , 3 sigmas: 4
